<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Auto_Combining_from_multiple_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [1]:
!pip install procgen

     |████████████████████████████████| 39.9MB 104kB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 3.3MB 42.9MB/s 
     |████████████████████████████████| 22.2MB 1.7MB/s 
     |████████████████████████████████| 204kB 46.4MB/s 
     |████████████████████████████████| 665kB 41.0MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [2]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

Cloning into 'my_project'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 573 (delta 67), reused 2 (delta 0), pack-reused 460
Receiving objects: 100% (573/573), 9.08 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (350/350), done.


In [3]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [4]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Mounted at /content/drive


# Import

In [5]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


In [6]:
print_list_of_experiments()

***** Experiment 0 *****
Description:    Baseline inspired by PPO article
***** Experiment 1 *****
Description:    Baseline inspired by Procgen article
***** Experiment 2 *****
Description:    Modified PPO baseline with value clipping enabled
***** Experiment 3 *****
Description:    Modified PPO baseline with value clipping enabled and learning rate given by par variable
***** Experiment 4 *****
Description:    Modified PPO baseline with value clipping enabled and reward penalty on death (1 as default)
***** Experiment 5 *****
Description:    Baseline inspired by IMPALA paper (No value clipping)
***** Experiment 6 *****
Description:    Inspired by IMPALA paper (With value clipping)
***** Experiment 7 *****
Description:    Inspired by both IMPALA and Procgen papers (With value clipping)
***** Experiment 8 *****
Description:    Inspired by both IMPALA and Procgen papers (With value clipping) and added death penalty


# Create Index File



In [7]:
INDEX = tools.create_index_table_from_txt_files()
INDEX

,total_steps,num_envs,num_levels,num_steps,num_epochs,batch_size,eps,grad_eps,value_coef,entropy_coef,lr,gamma,lmbda,version,time_limit,value_clipping,death_penalty,file_name,eval,eval_cycle,Time spent (in seconds),Steps taken,penalty,save_interval,step_start,Time spent,Last Mean Reward
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment0,43200,False,False,Experiment0_Run_26Nov_15h21m41s,True,16,18858.62,8003584,NaN,NaN,NaN,5:14:18,20.34375
Value,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0005,0.999,0.95,Experiment1,43200,False,False,Experiment1_Run_26Nov_15h22m38s,True,16,19818.86,8003584,NaN,NaN,NaN,5:30:18,19.53125
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment2,43200,True,False,Experiment2_Run_27Nov_11h05m48s,True,16,20480.86,8003584,NaN,NaN,NaN,5:41:20,28.15625
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment4,43200,True,True,Experiment4_Run_27Nov_11h07m20s,True,16,20819.16,8003584,NaN,NaN,NaN,5:46:59,25.8125
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.0025,0.99,0.95,Experiment3,43200,True,False,Experiment3_Run_27Nov_18h03m39s,True,16,NaN,4136960,NaN,NaN,NaN,NaN,23.0
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,0.00025,0.99,0.95,Experiment4,43200,True,True,Experiment4_Run_27Nov_18h11m00s,True,16,NaN,7077888,3,NaN,NaN,NaN,24.5
Value,8000000.0,32,10,256,3,32,0.2,0.5,0.5,0.01,0.0006,0.99,0.95,Experiment5,43200,False,False,Experiment5_Run_27Nov_16h37m42s,True,16,32130.90,8003584,NaN,NaN,NaN,8:55:30,16.28125
Value,8000000.0,32,10,256,3,32,0.2,0.5,0.5,0.01,0.0006,0.99,0.95,Experiment6,43200,True,False,Experiment6_Run_28Nov_12h09m43s,True,16,34241.34,8003584,1,NaN,NaN,9:30:41,24.15625
Value,8000000.0,32,10,256,3,512,0.1,0.5,1,0.01,2.5e-05,0.99,0.95,Experiment3,43200,True,False,Experiment3_Run_29Nov_16h30m27s,True,16,NaN,6979584,1,NaN,NaN,NaN,22.03125
Value,8000000.0,32,10,256,3,512,0.2,0.5,0.5,0.01,0.0006,0.99,0.95,Experiment7,43200,True,False,Experiment7_Run_30Nov_14h45m32s,True,16,27210.47,8003584,1,NaN,NaN,7:33:30,29.71875


In [8]:
import glob

file_names = [x.strip() for x in INDEX['file_name']]
n = len(file_names)
# for i in range(n):
#   file_names.append(INDEX['file_name'][-1-i].strip())

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plot_training_graphs(file_names, legend = None):
  df = pd.DataFrame(columns=file_names)
  df['Step'] = pd.read_csv(DATA_PATH + INDEX["file_name"][0].strip()+".csv")["Step"]  #This only works as long as they all have the same steps!
  for file_name in file_names:
    df[file_name] = pd.read_csv(DATA_PATH + file_name+".csv")[" Mean reward"]
  
  df = df.set_index('Step')

  plt.figure(figsize=(16, 10))
  sns.lineplot(data = df)
  if legend is not None:
    plt.legend(labels=[legend[x] for x in file_names])
  plt.title("Training")

def plot_eval_graphs(file_names, legend = None):
  df = pd.DataFrame(columns=file_names)
  for file_name in file_names:
    temp = pd.read_csv(DATA_PATH + file_name+"_EVAL.csv")
    df[file_name] = temp["avg"]
  plt.figure(figsize=(16, 10))
  sns.lineplot(data = df)
  if legend is not None:
    plt.legend(labels=[legend[x] for x in file_names])
  plt.title("Eval")
  # return df

In [10]:
for i in range(len(file_names)):
  print("[{:2d}]: {}".format(i, file_names[i]))

[ 0]: Experiment0_Run_26Nov_15h21m41s
[ 1]: Experiment1_Run_26Nov_15h22m38s
[ 2]: Experiment2_Run_27Nov_11h05m48s
[ 3]: Experiment4_Run_27Nov_11h07m20s
[ 4]: Experiment3_Run_27Nov_18h03m39s
[ 5]: Experiment4_Run_27Nov_18h11m00s
[ 6]: Experiment5_Run_27Nov_16h37m42s
[ 7]: Experiment6_Run_28Nov_12h09m43s
[ 8]: Experiment3_Run_29Nov_16h30m27s
[ 9]: Experiment7_Run_30Nov_14h45m32s
[10]: Experiment8_Run_30Nov_22h21m06s
[11]: Experiment7_100levels_Run_02Dec_09h29m48s
[12]: Experiment8_50levels_Run_02Dec_17h18m34s
[13]: Experiment8_10levels_Run_02Dec_09h48m26s
[14]: Experiment2_100levels_Run_03Dec_10h48m24s
[15]: Experiment8_50levels_Run_03Dec_14h53m09s
[16]: Experiment8_10levels_Run_03Dec_11h04m34s
[17]: Experiment8_50levels_Run_04Dec_06h17m22s
[18]: Experiment4_50levels_Run_04Dec_08h02m21s
[19]: Experiment2_50levels_Run_04Dec_18h10m32s
[20]: Experiment7_50levels_Run_04Dec_18h12m45s
[21]: Experiment8_50levels_Run_05Dec_08h34m02s
[22]: Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h

# Create .csv's

Assumes that all file_names are correct! (with step count etc)

In [21]:
def create_training_csv_from_multiple_runs(base_file_name):
  all_related_files = find_related_csv_files(base_file_name)

  #create a df for each file
  dfs = []
  for name in all_related_files:
    dfs.append(pd.read_csv(DATA_PATH + name + ".csv"))

  dfs = remove_overlap_from_dfs(dfs)

  #Concatenate
  final_df = pd.concat(dfs)
  final_df = final_df.set_index("Step")

  #Save to file
  final_df.to_csv(DATA_PATH +base_file_name +"_combined.csv")

def create_EVAL_csv_from_multiple_runs(base_file_name):
  all_related_files = find_related_csv_files(base_file_name)
  base = 32*256*16

  dfs = []
  for name in all_related_files:
    df = pd.read_csv(DATA_PATH + name + "_EVAL.csv")
    #add steps if missing
    if "step" not in df.columns:
      df.insert(0, "Step", [base*(i+1) for i in range(len(df))])
    else:
      df.rename(columns={'step':'Step'}, inplace=True)
    dfs.append(df)

  dfs = remove_overlap_from_dfs(dfs)

  #Remove whitespace in headers
  for df in dfs:
    df.rename(columns=lambda x: x.strip(), inplace=True)

    #Concatenate
  final_df = pd.concat(dfs)
  final_df = final_df.set_index("Step")

  #save in file
  final_df.to_csv(DATA_PATH +base_file_name +"_combined_EVAL.csv")


def remove_overlap_from_dfs(dfs):
  #Dataframes overlap, so cut off excess
  breakpoints=[]
  for df in dfs[1:]:
    breakpoints.append(df['Step'][0])
  for i in range(len(dfs)-1):
    dfs[i] = dfs[i][dfs[i]["Step"] < breakpoints[i]]
  return dfs

def find_related_csv_files(base_file_name):
  file_names = [x.strip() for x in INDEX['file_name']]
  all_related_files= []
  for name in file_names:
    if base_file_name in name and "combined" not in name:
      all_related_files.append(name)

  return all_related_files


In [22]:
base_file_name = "Experiment8_200levels_Run_06Dec_16h17m59s"
create_EVAL_csv_from_multiple_runs(base_file_name)
create_training_csv_from_multiple_runs(base_file_name)